In [1]:
# Get autocomplete to work
%config Completer.use_jedi = False

# Ensure external Python files are refreshed when reimporting things
%load_ext autoreload
%autoreload 2

In [7]:
from load_functions import load_text
    
text_input = load_text(day=8)
print(len(text_input))
text_input[:5]

200


['ecdbfag deacfb acdgb cdg acdbf gdfb efacdg gd cagdbf beacg | cdg dcebgaf gbdf bdacg',
 'fadecg gdbecaf agbfd fgdcb gab ebagdf feabcg deab gdefa ab | adfbg ab fcgdbae bfgecda',
 'cgebad edfagcb fg fedg ebfca gcefb fcedgb dbagcf cgf cdbeg | cfg acfbe bcgdafe dgeafcb',
 'bgcde cbefg gd dbeafc afbcgde bedgca gacd dbg cedba fbegda | agfcebd adgfbce dgb fgceb',
 'gadcbe gcade debfac fdagce egdbf cfedg fbgcade gafc dcf fc | fcga eacfdg gfca fdcbea']

Convert to list of signals and displays

In [4]:
signals_displays = []
for line in text_input:
    (signals, displays) = line.split(' | ')
    signals_displays.append([signals.split(), displays.split()])

signals_displays[0]

[['ecdbfag',
  'deacfb',
  'acdgb',
  'cdg',
  'acdbf',
  'gdfb',
  'efacdg',
  'gd',
  'cagdbf',
  'beacg'],
 ['cdg', 'dcebgaf', 'gbdf', 'bdacg']]

# Problem 

#### Part 1: Figure out how many 1s, 4s, 7s, and 8s there are

These consist of 2, 4, 3, and 7 segments respectively - and don't share this number of segments with any other digit

In [6]:
num_1s4s7s8s = 0

# We don't need to evaluate the inputs for this part of the problem
for (_, displays) in signals_displays:
    for d in displays:
        if len(d) in (2, 3, 4, 7):
            num_1s4s7s8s += 1

print('Day 8, part 1:', num_1s4s7s8s)

Day 6, part 1: 525


#### Part 2: Figure out exactly what digits are shown on the displays, and add up their values

In [75]:
'''
First, create dictionary to turn display inputs (correctly mapped) into output digits.

Inputs are shown as below

 aaaa  
b    c  
b    c  
 dddd   
e    f  
e    f  
 gggg 
 
So an input of 'acf' would result in a 7, an input of 'bcdf' would result in a 4, and so on.
'''
dict_digit_display = {
    'abcef':   0,
    'cf':      1,
    'acdeg':   2,
    'acdfg':   3,
    'bcdf':    4,
    'abdfg':   5,
    'abdefg':  6,
    'acf':     7,
    'abcdefg': 8,
    'abcdfg':  9,
    'abcefg':  0
}

# Used as possible segments a wire maps to
list_alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

# Then, figure out what the mapping of the input wires to the display (correct) segments is
def get_correct_mapping(signals):

    # First, sort each signal alphabetically and turn it into a list
    sorted_signals = [list(''.join(sorted(s))) for s in signals]
    
    # Then, sort this list by length
    # Returns a list with lengths 2, 3, 4, 5, 5, 5, 6, 6, 6, and 7
    len_sorted_signals = list(sorted(sorted_signals, key=len))
    
    # We know the top segment is on for a 7 (3 segments), and off for a 1 (2 segments)
    # So we can just check which wire is in the 3 length string and not the 2, and map that to segment a
    a_segment_wire = [alpha for alpha in len_sorted_signals[1] if alpha not in len_sorted_signals[0]][0]
    
    # Conversely, we know that the other 2 segments used in 1 & 7 must correspond to c & f
    # We don't know the order
    c_f_segment_wires = [alpha for alpha in len_sorted_signals[1] if alpha in len_sorted_signals[0]]
    
    # Then, we know that 4 has 4 segments, and that the wires for c & f have already been tagged above
    # So we are left with b & d wires
    b_d_segment_wires = [alpha for alpha in len_sorted_signals[2] if alpha not in len_sorted_signals[1]]
    
    # We know what wire segment a is, and for all 5 segment digits (2, 3, 5) we know that they all share 
    # 3 segments: a, d, and g. Therefore we can figure out what wires correspond to d & g
    d_g_segment_wires = [alpha for alpha in len_sorted_signals[3] if (alpha in len_sorted_signals[4]) and (alpha in len_sorted_signals[5])]
    d_g_segment_wires.remove(a_segment_wire)
    
    # Now, we can deduce what the b, d, and g segment wires are
    b_segment_wire = [alpha for alpha in b_d_segment_wires if alpha not in d_g_segment_wires][0]
    d_segment_wire = [alpha for alpha in b_d_segment_wires if alpha in d_g_segment_wires][0]
    g_segment_wire = [alpha for alpha in d_g_segment_wires if alpha not in b_d_segment_wires][0]
    
    # As we know a, b, d, and g. We can deduce which signal is 5 by the 5 segment signal that contains all 4 of these
    a_b_d_g_wires = [a_segment_wire, b_segment_wire, d_segment_wire, g_segment_wire]
    
    for five_segment_signal in len_sorted_signals[3:6]:
        assert len(five_segment_signal) == 5
        
        # Check how many of the segments match
        n_matches = 0
        for wire in five_segment_signal:
            if wire in a_b_d_g_wires:
                n_matches += 1
        
        # If 4 match, we know the remaining wire must map to segment f
        if n_matches == 4:
            f_segment_wire = [alpha for alpha in five_segment_signal if alpha not in a_b_d_g_wires][0]
            
    
    # Then, we know which remaining wire is c
    c_segment_wire = [alpha for alpha in c_f_segment_wires if alpha != f_segment_wire][0]
    
    # And finally, the last wire is e
    other_wires = [a_segment_wire, b_segment_wire, c_segment_wire, d_segment_wire, f_segment_wire, g_segment_wire]
    e_segment_wire = [alpha for alpha in list_alpha if alpha not in other_wires][0]
    
    # Write outputs
    dict_mapping = {
        a_segment_wire: 'a',
        b_segment_wire: 'b',
        c_segment_wire: 'c',
        d_segment_wire: 'd',
        e_segment_wire: 'e',
        f_segment_wire: 'f',
        g_segment_wire: 'g',
    }
        
    return dict_mapping


# Create function to get digits
def get_digits(signals, digits):
    
    # Get mapping of wires to segments
    dict_mapping = get_correct_mapping(signals)

    # Then convert each digit into an output digit
    output_digit_list = []
    for digit in digits:
        
        # Turn 'cb' into ['c', 'b']
        sorted_digit_wires = list(digit)
        
        # Look up what segment those inputs correspond to
        sorted_digit_segments = ''.join(sorted([dict_mapping[wire] for wire in sorted_digit_wires]))
        
        # And look up what digit it corresponds to
        output_digit_list.append(str(dict_digit_display[sorted_digit_segments]))
        
    return int(''.join(output_digit_list))


# Then add the digits up
result_pt2 = 0
for (signals, displays) in signals_displays:
    result_pt2 += get_digits(signals, displays)
    
print('Day 6, part 2:', result_pt2)


Day 6, part 2: 1083859
